#### Cargamos el dataset y normalizamos las columnas

#### Aplicación de K-MEANS al CRISIS
* En este notebook 'KMeans_general', analizamos el dataset general en base a los valores de las etiquetas de los programas presidenciales. PAra cada país, para cada año en que tuvo elecciones, se ponderó cada variable por el % de votos que sacó cada programa; de esta manera tenemos un registro por país por año.
* Dado que hay algunos países en que la suma de las variables 'per' no es 100%, se normalizaron de forma de que sumaran 100%
* A fin de contar con una buena masa de datos, sólo se considerá las variables per principales ('per' + 3 dígitos). En total 46 variables.
* Se aplicó PCA para reducir las dimensiones
* Aplicamos K-MEANS
* Hicimos análisis de los países-años en cada cluster

#### Conclusiones
* 
* 

In [6]:
from sklearn.cluster import KMeans

#### 1.Filtrar y Normalizar datos

In [ ]:
import pandas as pd
import numpy as np

# Cargar el dataset principal
file_path = './data/MP_Dataset_KMeans.xlsx'
df = pd.read_excel(file_path)

# Definir el rango temporal
desde = 2000  # Año de inicio
hasta = 2010  # Año de fin

# Seleccionar las variables asociadas al aspecto 'CRISIS_FINANCIERA_2008'
aspecto_filtro = 'CRISIS_FINANCIERA_2008'
variables_aspecto = df_aspectos[df_aspectos['aspecto'] == aspecto_filtro][['var_1', 'var_2', 'var_3', 'var_4']].dropna(axis=1).values.flatten()

# Filtrar las columnas relevantes y el rango temporal
columns_relevant = ['agno', 'countryname'] + list(variables_aspecto)
df_filtered = df[(df['agno'] >= desde) & (df['agno'] <= hasta)][columns_relevant].copy()

# Validar si las variables seleccionadas suman 1 y normalizar si es necesario
df_filtered['per_sum'] = df_filtered[variables_aspecto].sum(axis=1)
rows_to_normalize = df_filtered['per_sum'] != 1.0
df_filtered.loc[rows_to_normalize, variables_aspecto] = df_filtered.loc[rows_to_normalize, variables_aspecto].div(df_filtered.loc[rows_to_normalize, 'per_sum'], axis=0)
df_filtered.drop(columns=['per_sum'], inplace=True)

# Verificar la normalización
print(f"¿Todas las filas normalizadas correctamente? {df_filtered[variables_aspecto].sum(axis=1).round(6).eq(1).all()}")


#### 2.Realizar PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Escalar los datos
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_filtered[variables_aspecto])

# Aplicar PCA
pca = PCA(n_components=3)
pca_result = pca.fit_transform(data_scaled)

# Crear un DataFrame con los resultados de PCA
df_pca = pd.DataFrame(pca_result, columns=['PC1', 'PC2', 'PC3'])
df_pca['countryname'] = df_filtered['countryname']
df_pca['agno'] = df_filtered['agno']

# Visualizar varianza explicada
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, marker='o')
plt.title('Varianza explicada por componente principal')
plt.xlabel('Componente principal')
plt.ylabel('Proporción de varianza explicada')
plt.grid(alpha=0.3)
plt.show()


#### 3.Elbow + K-MEANS

In [ ]:
from sklearn.cluster import KMeans
from kneed import KneeLocator

# Calcular WCSS para determinar el número óptimo de clústeres
max_k = 10
wcss = []
for k in range(1, max_k + 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    wcss.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8, 6))
plt.plot(range(1, max_k + 1), wcss, marker='o')
plt.title('Método del codo')
plt.xlabel('Número de clústeres')
plt.ylabel('WCSS')
plt.grid(alpha=0.3)
plt.show()

# Encontrar el codo automáticamente
kl = KneeLocator(range(1, max_k + 1), wcss, curve="convex", direction="decreasing")
optimal_k = kl.knee
print(f"El número óptimo de clústeres según el método del codo es: {optimal_k}")


#### 4.Visualizar clusters

In [ ]:
# Aplicar K-Means con el número óptimo de clústeres
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df_pca['cluster'] = kmeans.fit_predict(data_scaled)

# Graficar los clústeres en 2D
plt.figure(figsize=(10, 8))
for cluster in range(optimal_k):
    cluster_data = df_pca[df_pca['cluster'] == cluster]
    plt.scatter(cluster_data['PC1'], cluster_data['PC2'], label=f'Cluster {cluster}', alpha=0.7)
plt.title(f'Clústeres para {aspecto_filtro} (2D)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Graficar los clústeres en 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
for cluster in range(optimal_k):
    cluster_data = df_pca[df_pca['cluster'] == cluster]
    ax.scatter(cluster_data['PC1'], cluster_data['PC2'], cluster_data['PC3'], label=f'Cluster {cluster}', alpha=0.7)
ax.set_title(f'Clústeres para {aspecto_filtro} (3D)')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.legend()
plt.show()


#### 5. Análisis de clusters

In [ ]:
# Revisar centroides
centroids = kmeans.cluster_centers_
print(f"Centroides de los clústeres:\n{pd.DataFrame(centroids, columns=variables_aspecto)}")

# Revisar países y años en cada clúster
clustered_data = pd.concat([df_filtered, df_pca[['cluster']]], axis=1)
for cluster in range(optimal_k):
    print(f"\n--- Cluster {cluster} ---")
    cluster_subset = clustered_data[clustered_data['cluster'] == cluster]
    print(f"Número de Programas: {len(cluster_subset)}")
    print(f"Países: {', '.join(cluster_subset['countryname'].unique())}")
    print(f"Años en este clúster: {', '.join(map(str, sorted(cluster_subset['agno'].unique())))}")
